In [1]:
import pandas as pd
import re
from tqdm import tqdm
import ftfy
import html
import os
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from torch.optim import AdamW

W0725 03:45:44.438000 3412 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
tqdm.pandas()
fusion_path= "d:/multiagent-disaster-reasoning/data/fusion_cleaned.csv"
df=pd.read_csv(fusion_path, encoding = 'utf-8')

In [20]:
def preprocess_tweet(text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)  # Remove non-ASCII
    text = re.sub(r"…|\.{3}", " ", text)        # Remove ellipses
    text = re.sub(r"[@＠]\w+", "@USER", text)   # Normalize mentions
    text = re.sub(r"(@USER\s*)+", "@USER ", text)  # Remove duplicate mentions
    text = re.sub(r"http\S+|www\S+", "http", text)
    text = re.sub(r"(http\s+)+", "http ", text)     # Remove duplicate http
    text = re.sub(r"#", "", text)                  # Remove # but keep tag
    text = re.sub(r"\s+", " ", text).strip()       # Normalize spaces
    return text


df["tweet_text_clean"] = df["tweet_text"].progress_apply(preprocess_tweet)

output_path = "d:/multiagent-disaster-reasoning/data/cleaned_fusion_with_text.csv"
df.to_csv(output_path, index = False)
print("csv saved")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8534/8534 [00:00<00:00, 19200.76it/s]


csv saved


In [3]:
CSV_PATH = "d:/multiagent-disaster-reasoning/data/cleaned_fusion_with_text.csv"
MODEL_NAME = "vinai/bertweet-base"
BATCH_SIZE = 16
MAX_LEN = 128
EPOCHS = 5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SAVE_PATH = "d:/multiagent-disaster-reasoning/agents/text_agent_bertweet.pth"

In [ ]:
df = pd.read_csv(CSV_PATH)
df = df[['tweet_text_clean', 'disaster_type']].dropna()

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['disaster_type'])
num_labels = len(label_encoder.classes_)


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast = True)

class TweetDataset(Dataset):
    def __init__(self, texts, labels):
        self.labels = list(labels)
        self.encodings = tokenizer(list(texts), truncation = True, padding = 'max_length', max_length = MAX_LEN, return_tensors =None)
        if len(self.encodings['input_ids']) != len(self.labels):
            raise ValueError("Mismatch between encodings and labels")
        

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_texts, val_texts, train_labels, val_labels = train_test_split(df['tweet_text_clean'], df['label'], test_size =0.1, stratify = df['label'], random_state =42)
train_texts = train_texts.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)
val_texts = val_texts.reset_index(drop=True)
val_labels = val_labels.reset_index(drop=True)

train_dataset = TweetDataset(train_texts, train_labels)
val_dataset = TweetDataset(val_texts, val_labels)

class_counts = df['label'].value_counts().sort_index().values
weights = 1. / class_counts
sample_weight = weights[train_labels]
sampler = WeightedRandomSampler(sample_weight, len(sample_weight))

train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, sampler = sampler)
val_loader = DataLoader(val_dataset, batch_size = BATCH_SIZE)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels= num_labels)
model.to(DEVICE)
optimizer = AdamW(model.parameters(), lr = 2e-5)

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc = f"Epoch {epoch+1}/{EPOCHS} - Training"):
        batch = {k: v.to(DEVICE) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch+1} Train Loss: {total_loss/len(train_loader): .4f}")

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc = f"Epoch {epoch+1}/{EPOCHS} - Validataion"):
            batch = {k: v.to(DEVICE) for k, v in batch.items()}
            outputs = model(**batch)
            preds = outputs.logits.argmax(dim=1)
            correct += (preds == batch['labels']).sum().item()
            total += batch['labels'].size(0)
    print(f"Validation Accuracy: {correct/total: .4f}")


torch.save({
    'model_state_dict': model.state_dict(),
    'label_encoder': label_encoder,
    'tokenizer': tokenizer,
}, SAVE_PATH)

print("Model saved at path", SAVE_PATH)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/5 - Training:   0%|▎                                                           | 2/480 [00:14<57:27,  7.21s/it]